In [1]:
import os
import datetime
import pandas as pd
import re
import great_expectations as gx
import agriculture as ag

In [2]:
#!pip install great_expectations

## Descargando los datos

Para descargar los datos referentes a los precios de diversos productos agricolas, podemos ejecutar directamente el script `agriculture.py` en la terminal de la siguiente manera:

```python3 agriculture.py```

El script mencionado llama de forma iterativa a la API de la pagina del Sistema Nacional de Informacion de Mercados (SNIIM) de la Secretaria de Economia, para recuperar los precios de diversos productos en diversos centros de distribucion en el pais, desde el año 2020 hasta la fecha.

Otra manera de descargar los datos es ejecutando la siguiente celda, la cual inicializa un objeto de tipo `ScrapperMarketAgriculture`, el cual contiene la funcion `scraping`, la cual realiza el mismo proceso del script mencionado anteriormente.

In [3]:
# ESTE SCRAPPER PUEDE TARDAR, RECOMENDABLE EJECUTAR DIRECTAMENTE DESDE TERMINAL
scrapper = ag.ScrapperMarketAgriculture() # Generamos un objeto de tipo ScrapperMarketAgriculture (scrapper SNIIM)
scrapper.scraping() # Scrapear la pagina del SNIIM para las frutas y hortalizas

Producto: Acelga - Primera
Producto: Aguacate Criollo - Primera
Producto: Aguacate Fuerte - Primera
Producto: Aguacate Hass - Calidad extra
Producto: Aguacate Hass - Calidad super extra
Producto: Aguacate Hass - Primera
Producto: Aguacate Hass adelantado - Primera
Producto: Aguacate Hass flor vieja - Primera
Producto: Aguacate Pagua - Primera
Producto: Ajo Blanco - Primera
Producto: Ajo Blanco # 8 - Primera
Producto: Ajo Morado - Primera
Producto: Apio - Primera
Producto: Berenjena - Primera
Producto: Betabel - Primera
Producto: Brócoli - Primera
Producto: Cacahuate - Primera
Producto: Calabacita Criolla - Primera
Producto: Calabacita Italiana - Primera
Producto: Calabacita regional - Primera
Producto: Calabaza de castilla - Primera
Producto: Camote - Primera
Producto: Caña - Primera
Producto: Cebolla Bola - Primera
Producto: Cebolla Bola grande - Primera
Producto: Cebolla de rabo - Primera
Producto: Cebolla Morada - Primera
Producto: Cilantro - Primera
Producto: Ciruela Amarilla - Pri

Una vez recuperados los datos, estos se almacenan de forma local en archivos .csv, los cuales se encuentran en el directorio `data/sniim`

In [4]:
# Directorio que contiene los archivos generados por el scrapper
sniim_dir = "./data/sniim/"

## Procesamiento de los datos

Como al final tendremos una dataframe conteniendo los precios de diversos productos, podemos tratar de obtener el nombre de cada producto a partir del nombre del archivo csv, para esto definimos la siguiente funcion:

In [5]:
# Definimos una funcion para obtener el nombre del cultivo a partir del nombre del archivo csv
def estandarizar_nombre(nombre : str) -> str:
    """
    Regresa el nombre del cultivo/producto a partir del nombre del archivo .csv
    """
    cultivo_std = nombre.split('_')[:-1]
    cultivo_std = '_'.join(cultivo_std)
    cultivo_std = re.sub(r"(?<=#)_(?=\d)",'',cultivo_std)
    cultivo_std = re.sub("_"," ",cultivo_std)
    cultivo_std = re.sub("csemilla", "con semilla", cultivo_std)
    cultivo_std = re.sub("ssemilla", "sin semilla", cultivo_std)
    cultivo_std = cultivo_std.title()

    return cultivo_std

Finalmente, podemos iterar sobre los archivos presentes en el directorio anterior, para generar una dataframe por cada uno de ellos, agregar las dataframes a una lista para posteriormente concatenarlas en una sola.

Ademas, a cada dataframe le agregamos la columna `producto` para poder diferenciar las entradas una vez concatenadas las dataframes, y separamos la columna `destino` en dos: `destino` y `central` para identificar mas facilmente la Entidad de destino y la central de abastos donde se levanto la muestra.

In [6]:
# Almacenaremos nuestras dataframes en una lista para despues concatenar
df_list = []
for dirpath, subdir, files in os.walk(sniim_dir):
    for file in files:
        if file == ".DS_Store":
            continue
        
        df = pd.read_csv(os.path.join(dirpath,file)) 
        df["producto"] = estandarizar_nombre(file) # Obtenemos el nombre del producto a partir del nombre del archivo
        df[["destino","central"]] = df["destino"].str.split(": ", expand=True) # Separamos la columna destino en 2
        
        # Reordenamos las columnas
        cols = df.columns.to_list()
        del cols[-1]
        cols.insert(4,"central")
        
        del cols[-1]
        cols.insert(1,"producto")
        
        df = df[cols]
        
        # Agregamos la dataframe generada a la lista
        df_list.append(df)

In [7]:
precios_sniim = pd.concat(df_list,ignore_index=True) # Concatenamos nuestra lista de dataframes en una sola

Una vez obtenida nuestra dataframe final, podemos utilizar el metodo `sample` para obtener una vista general de nuestra dataframe.

In [8]:
precios_sniim.sample(10)

,fecha,producto,presentacion,origen,destino,central,precio_min,precio_max,precio_frec,obs
336025,04/11/2021,Elote Grande,Pieza,Nuevo León,Nuevo León,"Mercado de Abasto ""Estrella"" de San Nicolás de...",4.0,6.0,5.00,NaN
993190,25/06/2021,Yerbabuena,Rollo,Yucatán,Yucatán,Central de Abasto de Mérida,60.0,60.0,60.00,NaN
1125681,12/04/2022,Calabacita Italiana,Caja de 28 kg.,México,México,Central de Abasto de Toluca,8.93,10.0,8.93,NaN
1248100,14/12/2020,Pepino,Kilogramo,Chiapas,Chiapas,Central de Abasto de Tuxtla Gutiérrez,8.0,10.0,8.00,NaN
1809147,25/01/2023,Manzana Golden Delicious,Kilogramo,Nayarit,Nayarit,Nayarabastos de Tepic,70.0,72.0,71.00,NaN
1064928,30/12/2021,Coliflor Mediana,Caja de 16 pzas.,Baja California,Baja California Sur,Unión de Comerciantes de La Paz,23.75,34.38,34.38,NaN
1492750,06/01/2023,Nopal,Caja de 20 kg.,Aguascalientes,San Luis Potosí,Centro de Abasto de San Luis Potosí,12.5,13.0,12.50,NaN
829266,30/11/2021,Nopal Grande,Caja de 35 kg.,Distrito Federal,Tabasco,Central de Abasto de Villahermosa,9.14,10.0,9.14,NaN
278588,16/05/2023,Chayote Sin Espinas,Kilogramo,Jalisco,Chihuahua,Mercado de Abasto de Cd. Juárez,14.0,14.0,14.00,NaN
1168515,15/09/2020,Betabel,Kilogramo,Veracruz,Veracruz,Central de Abasto de Jalapa,6.0,6.5,6.00,NaN


Para una mayor facilidad al momento de comprender nuestra dataframe, generamos un diccionario de datos para esta, explicando lo que representa cada una de las columnas.

In [9]:
precios_dict = {
    "fecha": "Fecha en la que se levanto la encuesta",
    "product": "Producto al que hace referencia la entrada/muestra",
    "presentacion": "Presentacion del producto al que se hace referencia",
    "origen": "Entidad Federativa de donde proviene el producto",
    "destino": "Entidad Federativa hacia donde llego el producto",
    "central": "Central de abastos/centro de distribucion de donde se obtuvo la informacion",
    "precio_min": "El valor más bajo de la cotización dentro de una muestra (MXN)",
    "precio_max": "El valor más alto de la cotización dentro de una muestra (MXN)",
    "precio_frec": "Es el dato que más se repite en la muestra (moda) (MXN)",
    "obs": "Observaciones encontradas para la muestra"
}

In [10]:
dict_df = pd.DataFrame.from_dict(precios_dict,orient="index")
dict_df

,0
fecha,Fecha en la que se levanto la encuesta
product,Producto al que hace referencia la entrada/mue...
presentacion,Presentacion del producto al que se hace refer...
origen,Entidad Federativa de donde proviene el producto
destino,Entidad Federativa hacia donde llego el producto
central,Central de abastos/centro de distribucion de d...
precio_min,El valor más bajo de la cotización dentro de u...
precio_max,El valor más alto de la cotización dentro de u...
precio_frec,Es el dato que más se repite en la muestra (mo...
obs,Observaciones encontradas para la muestra


## Limpieza de los datos

Una vez generado nuestro diccionario, podemos comenzar a tratar los datos de nuestro dataframe. Algo que podemos observar del sample de nuestro dataframe es que la columna `obs` parece ser la unica que cuenta con valores nulos. Podemos tener una idea de la proporcion de datos faltantes en esta columna ejecutando la siguiente celda, la cual calcula el porcentaje de entradas faltantes en la misma.

In [11]:
print(f"Porcentaje de entradas faltantes en columna 'obs': {100* precios_sniim['obs'].isna().sum() / precios_sniim.shape[0]:.2f}%")

Porcentaje de entradas faltantes en columna 'obs': 96.03%


Vemos que la mayoria de las entradas en nuestra dataframe (> 90%) no cuenta con informacion en la columna `obs`, por lo que puede ser mas conveniente deshacernos de esta columna.

In [12]:
precios_sniim.drop(columns="obs", inplace=True)

In [13]:
precios_sniim.sample(10)

,fecha,producto,presentacion,origen,destino,central,precio_min,precio_max,precio_frec
1495333,06/10/2022,Papa Alpha,Kilogramo,Sinaloa,Coahuila,"Central de Abasto de La Laguna, Torreón",25.0,29.0,28.00
196831,10/03/2021,Apio,Caja de 25 kg.,Guanajuato,Jalisco,Mercado de Abasto de Guadalajara,12.0,12.4,12.00
763777,15/09/2022,Cilantro,Rollo de 5 kg.,Puebla,Veracruz,Central de Abasto de Jalapa,20.0,22.0,22.00
348095,12/05/2021,Naranja Valencia Chica,Kilogramo,Veracruz,México,Central de Abasto de Ecatepec,7.0,9.0,7.00
467120,29/08/2023,Naranja Valencia Mediana,Caja de 18 kg.,Sonora,Sonora,"Mercado de Abasto ""Francisco I. Madero"" de Her...",22.22,23.33,23.06
1943667,07/06/2021,Manzana Red Delicious,Kilogramo,Importación,Morelos,"Mercado ""Adolfo López Mateos"" de Cuernavaca",38.0,40.0,40.00
1003599,10/03/2020,Lechuga Romanita Grande,Docena,Guanajuato,Durango,"Central de Abasto ""Francisco Villa""",9.0,10.0,10.00
1287401,05/04/2021,Mango Oro,Caja de 30 kg.,Oaxaca,Yucatán,Centro Mayorista Oxkutzcab,8.33,8.33,8.33
1094433,01/04/2022,Limon Sin Semilla,Kilogramo,Quintana Roo,Quintana Roo,"Mercado de Chetumal, Quintana Roo",80.0,80.0,80.00
396195,30/12/2021,Jamaica,Kilogramo,Puebla,Veracruz,Mercado Malibrán,120.0,135.0,120.00


Posteriormente, podemos utilizar el metodo `info` de nuestra dataframe para obtener informacion acerca de los tipos de datos de las columnas o la posible presencia de mas datos faltantes.

In [14]:
precios_sniim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2019574 entries, 0 to 2019573
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   fecha         object 
 1   producto      object 
 2   presentacion  object 
 3   origen        object 
 4   destino       object 
 5   central       object 
 6   precio_min    object 
 7   precio_max    object 
 8   precio_frec   float64
dtypes: float64(1), object(8)
memory usage: 138.7+ MB


Lo primero que notamos es que la columna `fecha` tiene un tipo de dato `object`, lo cual puede significar que esta almacenada como un string. Al tratarse de datos temporales, podemos convertir esta columna a un objeto de tipo `datetime` para una mayor facilidad al momento de manipular o explorar los datos.

In [15]:
precios_sniim["fecha"] = pd.to_datetime(precios_sniim["fecha"])

Despues podemos analizar los valores en la columna `origen` para verificar que no haya valores inesperados. Los posibles valores en esta columna son los nombres de los 32 estados de Mexico, o bien "Importacion" si el producto fue importado del extranjero, o "Nacional" si no hay datos sobre su estado de origen.

In [16]:
precios_sniim["origen"].unique()

array(['Nayarit', 'Distrito Federal', 'Oaxaca', 'Importación', 'Sonora',
       'Yucatán', 'Baja California', 'Campeche', 'San Luis Potosí',
       'Chiapas', 'Durango', 'Hidalgo', 'Michoacán', 'Jalisco',
       'Guanajuato', 'México', 'Quintana Roo', 'Sinaloa', 'Puebla',
       'Nuevo León', 'Aguascalientes', 'Zacatecas', 'Tamaulipas',
       'Chihuahua', 'Coahuila', 'Guerrero', 'Colima', 'Veracruz',
       'Querétaro', 'Tabasco', 'Morelos', 'Baja California Sur',
       'Nacional', 'Sin Especificar'], dtype=object)

Vemos que no existen anomalias aparentes en esta columna, asi que podemos proceder a revisar los valores de la columna `destino`, la cual deberia tener valores similares a la columna anterior.

In [17]:
precios_sniim["destino"].unique()

array(['Nayarit', 'Oaxaca', 'Campeche', 'DF', 'México', 'Yucatán',
       'Baja California ', 'Coahuila', 'Chiapas', 'Durango', 'Guanajuato',
       'Guerrero', 'Jalisco', 'Michoacán', 'Nuevo León', 'Querétaro',
       'Quintana Roo', 'San Luis Potosí', 'Sinaloa', 'Sonora',
       'Tamaulipas', 'Veracruz', 'Aguascalientes', 'Zacatecas', 'Morelos',
       'Hidalgo', 'Puebla', 'Tabasco', 'Baja California Sur', 'Chihuahua',
       'Colima'], dtype=object)

Vemos que tampoco hay anomalias en esta columna. Sin embargo, algunos valores contienen espacios al final del valor (por ejemplo, `Baja California `). Esto se puede corregir facilmente utilizando la funcion `strip` que nos ofrece pandas:

In [18]:
precios_sniim["destino"] = precios_sniim["destino"].str.strip()
precios_sniim["destino"].unique()

array(['Nayarit', 'Oaxaca', 'Campeche', 'DF', 'México', 'Yucatán',
       'Baja California', 'Coahuila', 'Chiapas', 'Durango', 'Guanajuato',
       'Guerrero', 'Jalisco', 'Michoacán', 'Nuevo León', 'Querétaro',
       'Quintana Roo', 'San Luis Potosí', 'Sinaloa', 'Sonora',
       'Tamaulipas', 'Veracruz', 'Aguascalientes', 'Zacatecas', 'Morelos',
       'Hidalgo', 'Puebla', 'Tabasco', 'Baja California Sur', 'Chihuahua',
       'Colima'], dtype=object)

Finalmente, la columna `precio_max` aparece como de tipo `object`, por lo que posiblemente hay valores que impidieron que fuera parseada a `float`.

Despues de una inspeccion de esta columna, vemos que para algunos precios, se utiliza la coma (,) para separar los millares. Podemos eliminar esta coma de nuestro dataframe con el metodo `replace`, para despues transformarla a tipo `float` con el metodo `astype(float)`.

In [19]:
precios_sniim["precio_max"] = precios_sniim["precio_max"].astype(str).str.replace(",","")
precios_sniim["precio_max"] = precios_sniim["precio_max"].astype(float)

In [20]:
precios_sniim["precio_min"] = precios_sniim["precio_min"].astype(str).str.replace(",","")
precios_sniim["precio_min"] = precios_sniim["precio_min"].astype(float)

Finalmente, podemos volver a obtener una muestra de nuestro dataframe para ver su estructura final, asi como volver a llamar al metodo `info` para verificar los tipos de datos de cada columna y que no haya valores faltantes.

In [21]:
precios_sniim.sample(10)

,fecha,producto,presentacion,origen,destino,central,precio_min,precio_max,precio_frec
433872,2021-10-26,Brocoli,Kilogramo,Guanajuato,Nuevo León,"Mercado de Abasto ""Estrella"" de San Nicolás de...",30.00,35.00,32.00
532744,2022-03-16,Piña Mediana,Pieza,Veracruz,DF,Central de Abasto de Iztapalapa DF,27.00,30.00,28.00
1965430,2023-02-13,Chile De Arbol Seco,Kilogramo,Puebla,Veracruz,Central de Abasto de Minatitlán,150.00,165.00,160.00
1184265,2021-09-09,Papaya Maradol,Kilogramo,Chiapas,Jalisco,Mercado Felipe Ángeles de Guadalajara,24.50,24.50,24.50
1971451,2020-04-28,Manzana Red Delicious,Caja de 18 kg.,Importación,Nuevo León,"Mercado de Abasto ""Estrella"" de San Nicolás de...",25.00,30.56,28.89
317246,2022-07-12,Chayote Sin Espinas,Caja de 28 kg.,Jalisco,Jalisco,Mercado de Abasto de Guadalajara,4.29,4.64,4.29
200064,2021-07-05,Apio,Caja de 24 kg.,Nuevo León,Tamaulipas,Módulo de Abasto de Reynosa,12.92,13.33,13.33
1994390,2023-02-22,Champiñon,Kilogramo,Querétaro,Querétaro,Mercado de Abasto de Querétaro,85.00,90.00,90.00
1212944,2020-07-01,Tomate Saladette,Kilogramo,Puebla,Guerrero,Central de Abastos de Acapulco,12.00,12.00,12.00
630058,2022-06-27,Ciruela Roja,Caja de 10 kg.,Importación,Coahuila,"Central de Abasto de La Laguna, Torreón",50.00,64.00,60.00


In [22]:
precios_sniim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2019574 entries, 0 to 2019573
Data columns (total 9 columns):
 #   Column        Dtype         
---  ------        -----         
 0   fecha         datetime64[ns]
 1   producto      object        
 2   presentacion  object        
 3   origen        object        
 4   destino       object        
 5   central       object        
 6   precio_min    float64       
 7   precio_max    float64       
 8   precio_frec   float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 138.7+ MB


## Validacion de los datos

In [23]:
precios_sniim_2 = gx.from_pandas(precios_sniim)

In [24]:
type(precios_sniim_2)

great_expectations.dataset.pandas_dataset.PandasDataset

In [25]:
precios_sniim_2.info()

<class 'great_expectations.dataset.pandas_dataset.PandasDataset'>
RangeIndex: 2019574 entries, 0 to 2019573
Data columns (total 9 columns):
 #   Column        Dtype         
---  ------        -----         
 0   fecha         datetime64[ns]
 1   producto      object        
 2   presentacion  object        
 3   origen        object        
 4   destino       object        
 5   central       object        
 6   precio_min    float64       
 7   precio_max    float64       
 8   precio_frec   float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 138.7+ MB


In [42]:
precios_sniim_2.expect_column_values_to_be_between(column="precio_frec",min_value=0,max_value=1000)

{
  "success": true,
  "result": {
    "element_count": 2019574,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [ ]:
precios_sniim_2.exp